In [5]:
# input a path to download and unzip files:
import os
os.chdir(r'G:\My Drive\Teaching\OATP\E1\FormAP')
path=r"G:\My Drive\Teaching\OATP\E1\FormAP"

In [6]:
#imprting required libraries
import pandas as pd
import numpy as np
import requests
import os
import zipfile
import jupytab
import json
from urllib.request import urlopen

In [7]:
path2=path+"\\Firm Filings.zip"
path3=path+"\\Firm Filings"

In [ ]:
#function to download the file

def download_url(url, save_path, chunk_size=128):
    r = requests.get(url, stream=True)
    with open(save_path, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=chunk_size):
            fd.write(chunk)
download_url("https://pcaobus.org/assets/PCAOBFiles/FirmFilings.zip",path2)

In [ ]:
#unzip the file
with zipfile.ZipFile(path2, 'r') as zip_ref:
    zip_ref.extractall(path3)

In [ ]:
form_aps=pd.read_csv(path3+"\FirmFilings.csv",parse_dates=['Signed Date','Audit Report Date','Fiscal Period End Date','Filing Date'], low_memory=False)

In [ ]:
# excluding amendment filings
form_aps2=form_aps[((form_aps['Amendment Firm Identification']==False) | (form_aps['Amendment Firm Identification'].isnull()))
                   & ((form_aps['Amendment Audit Report']==False) | (form_aps['Amendment Audit Report'].isnull()) )
                   & ((form_aps['Amendment Other Firms']==False) | (form_aps['Amendment Other Firms'].isnull()) )
                   & ((form_aps['Amendment Divided Responsibility']==False) | (form_aps['Amendment Divided Responsibility'].isnull()))
                   & ((form_aps['Amendment Participants > 5%']==False) | (form_aps['Amendment Participants > 5%'].isnull()) )
                   & ((form_aps['Amendment Participants < 5%']==False) | (form_aps['Amendment Participants < 5%'].isnull()) )
                   & ((form_aps['Amendment Audit Divided']==False) | (form_aps['Amendment Audit Divided'].isnull() ) )
                   & ((form_aps['Amendment Certified Firm']==False) | (form_aps['Amendment Certified Firm'].isnull())) 
                     & (form_aps['Is Multiple Audit Period']==False ) & (form_aps['Is Audit Divided']==False)]


In [ ]:
#non missing ciks
form_aps2=form_aps2[form_aps2['Issuer CIK'].notnull()]

In [ ]:
form_aps2['Issuer CIK'] = form_aps2['Issuer CIK'].apply(lambda x: str(int(x)).zfill(10))

In [ ]:
form_aps2=form_aps2[['Form Filing ID','Firm ID','Firm Name','Firm Country',
                              'Issuer CIK','Audit Report Date','Fiscal Period End Date',
                              'Engagement Partner Last Name','Engagement Partner First Name',
                              'Engagement Partner Middle Name','Engagement Partner Suffix','Engagement Partner ID',
                              'Firm Issuing Country','Firm Issuing City','Firm Issuing State','Filing Date']]
form_aps2.reset_index(drop=True,inplace=True)
form_aps2['Form Filing ID']=form_aps2['Form Filing ID'].astype(str)
form_aps2['Firm ID']=form_aps2['Firm ID'].astype(str)


In [ ]:
tables = jupytab.Tables()
tables['FormAP_Filings'] = jupytab.DataFrameTable('Form AP Filings', form_aps2,include_index=False)


In [ ]:
# GET /schema
tables.render_schema()

In [ ]:
# GET /data
try:
    tables.render_data(REQUEST)
except NameError:
    print("Not available outside jupytab context")